In [1]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Dropout, Flatten, Dense

from art.attacks.evasion import BasicIterativeMethod
from art.estimators.classification import TensorFlowV2Classifier
from art.utils import load_dataset

import numpy as np

from sklearn import metrics

import joblib

2024-02-26 01:15:57.302316: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-02-26 01:15:57.305216: I external/local_tsl/tsl/cuda/cudart_stub.cc:31] Could not find cuda drivers on your machine, GPU will not be used.
2024-02-26 01:15:57.336846: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-02-26 01:15:57.336902: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-02-26 01:15:57.337945: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to

In [2]:
def calculate_performance_metrics(x_test, y_test, model):

    # Predictions
    y_pred = model.predict(x_test)
    y_pred_classes = np.argmax(y_pred, axis=1)
    
    report = metrics.classification_report(y_test, y_pred_classes, output_dict=True, zero_division=1)
    
    precision_macro = report['macro avg']['precision']
    precision_weighted = report['weighted avg']['precision']
    accuracy = report['accuracy']
    recall_macro = report['macro avg']['recall']
    recall_weighted = report['weighted avg']['recall']
    f1_macro = report['macro avg']['f1-score']
    f1_weighted = report['weighted avg']['f1-score']
    auc = metrics.roc_auc_score(y_test, y_pred, multi_class='ovr')

    print(f"Accuracy: {accuracy}\nAUC: {auc}")
    
    print("\nmacro")
    print(f"Precision: {precision_macro}\nRecall: {recall_macro}\nF1 Score: {f1_macro}")

    print("\nweighted")
    print(f"Precision: {precision_weighted}\nRecall: {recall_weighted}\nF1 Score: {f1_weighted}")
    print()
    
    # Confusion matrix for FNR, TNR, FPR, TPR
    cm = metrics.confusion_matrix(y_test, y_pred_classes)
    def calculate_rates(conf_matrix, class_index):
        tp = conf_matrix[class_index, class_index]
        fn = np.sum(conf_matrix[class_index, :]) - tp
        fp = np.sum(conf_matrix[:, class_index]) - tp
        tn = np.sum(conf_matrix) - (tp + fn + fp)
    
        fnr = fn / (fn + tp)
        tnr = tn / (tn + fp)
        fpr = fp / (fp + tn)
        tpr = tp / (tp + fn)
        return fnr, tnr, fpr, tpr

    # Calculate and aggregate rates
    fnrs, tnrs, fprs, tprs = [], [], [], []
    for i in range(cm.shape[0]):
        fnr, tnr, fpr, tpr = calculate_rates(cm, i)
        fnrs.append(fnr)
        tnrs.append(tnr)
        fprs.append(fpr)
        tprs.append(tpr)
    
    mean_fnr = np.mean(fnrs)
    mean_tnr = np.mean(tnrs)
    mean_fpr = np.mean(fprs)
    mean_tpr = np.mean(tprs)

    # Printing the mean metrics
    print(f"Mean FNR: {mean_fnr}\nMean TNR: {mean_tnr}\nMean FPR: {mean_fpr}\nMean TPR: {mean_tpr}")

In [3]:
x_test = np.load("/home/jovyan/X-IIoT/X-IIoT_preprocessed/x_test.npy")
x_train = np.load("/home/jovyan/X-IIoT/X-IIoT_preprocessed/x_train.npy")
x_val = np.load("/home/jovyan/X-IIoT/X-IIoT_preprocessed/x_val.npy")
y_test = np.load("/home/jovyan/X-IIoT/X-IIoT_preprocessed/y_test.npy")
y_train = np.load("/home/jovyan/X-IIoT/X-IIoT_preprocessed/y_train.npy")
y_val = np.load("/home/jovyan/X-IIoT/X-IIoT_preprocessed/y_val.npy")

In [4]:
model = joblib.load("/home/jovyan/X-IIoT/dl/dl_new/model/dnn.joblib")

2024-02-26 01:16:03.595122: W tensorflow/core/util/tensor_slice_reader.cc:98] Could not open ram://1f7fc854fb404807ab5bc66cd02f69a6: INVALID_ARGUMENT: ram://1f7fc854fb404807ab5bc66cd02f69a6 is a directory.


In [5]:
classifier = TensorFlowV2Classifier(
    model=model,
    nb_classes=19,
    input_shape=(58,),
    loss_object=tf.keras.losses.SparseCategoricalCrossentropy(),
)

In [6]:
# Initialize the Elastic Net attack
attack = BasicIterativeMethod(estimator=classifier, eps=0.1)

# Generate adversarial test examples
x_test_adv = attack.generate(x=x_test)

In [7]:
calculate_performance_metrics(x_test_adv, y_test, model)

4383/4383 [==============================] - 3s 578us/step
Accuracy: 0.2829850959138558
AUC: 0.6678050706456131

macro
Precision: 0.19696619036899507
Recall: 0.09417994161085466
F1 Score: 0.07749355654933594

weighted
Precision: 0.3431187603796154
Recall: 0.2829850959138558
F1 Score: 0.3020850568918105

Mean FNR: 0.9058200583891454
Mean TNR: 0.9446008365549259
Mean FPR: 0.05539916344507407
Mean TPR: 0.09417994161085466


In [7]:
# Initialize the Elastic Net attack
attack = BasicIterativeMethod(estimator=classifier, eps=0.15)

# Generate adversarial test examples
x_test_adv = attack.generate(x=x_test)

In [8]:
calculate_performance_metrics(x_test_adv, y_test, model)

4383/4383 [==============================] - 3s 586us/step
Accuracy: 0.11727875632888826
AUC: 0.5402252572165684

macro
Precision: 0.14877687907102305
Recall: 0.04197606194491521
F1 Score: 0.0357596372735205

weighted
Precision: 0.18998974026237836
Recall: 0.11727875632888826
F1 Score: 0.14028347449813394

Mean FNR: 0.9580239380550847
Mean TNR: 0.9355652389956722
Mean FPR: 0.0644347610043277
Mean TPR: 0.04197606194491521


In [9]:
# Initialize the Elastic Net attack
attack = BasicIterativeMethod(estimator=classifier, eps=0.2)

# Generate adversarial test examples
x_test_adv = attack.generate(x=x_test)

In [10]:
calculate_performance_metrics(x_test_adv, y_test, model)

4383/4383 [==============================] - 3s 583us/step
Accuracy: 0.05160094131070384
AUC: 0.46846056198429503

macro
Precision: 0.1256539536455159
Recall: 0.02258123415217975
F1 Score: 0.017419574384132986

weighted
Precision: 0.09673833648721825
Recall: 0.05160094131070384
F1 Score: 0.062146588476941725

Mean FNR: 0.9774187658478204
Mean TNR: 0.9332030607464389
Mean FPR: 0.06679693925356105
Mean TPR: 0.02258123415217975


In [11]:
# Initialize the Elastic Net attack
attack = BasicIterativeMethod(estimator=classifier, eps=0.25)

# Generate adversarial test examples
x_test_adv = attack.generate(x=x_test)

In [12]:
calculate_performance_metrics(x_test_adv, y_test, model)

4383/4383 [==============================] - 3s 580us/step
Accuracy: 0.026050060614704416
AUC: 0.42919190202287527

macro
Precision: 0.11575540428181848
Recall: 0.014590880103849594
F1 Score: 0.01004607537523594

weighted
Precision: 0.04811212656086451
Recall: 0.026050060614704416
F1 Score: 0.029563901468056746

Mean FNR: 0.9854091198961503
Mean TNR: 0.9318721708848697
Mean FPR: 0.0681278291151302
Mean TPR: 0.014590880103849594


In [6]:
# Initialize the Elastic Net attack
attack = BasicIterativeMethod(estimator=classifier, eps=0.3)

# Generate adversarial test examples
x_test_adv = attack.generate(x=x_test)

In [7]:
calculate_performance_metrics(x_test_adv, y_test, model)

4383/4383 [==============================] - 3s 589us/step
Accuracy: 0.016651215859659133
AUC: 0.3462443276953082

macro
Precision: 0.057684522512523866
Recall: 0.019883735906578694
F1 Score: 0.006975360772884151

weighted
Precision: 0.015167758877872695
Recall: 0.016651215859659133
F1 Score: 0.010977232711232987

Mean FNR: 0.9801162640934213
Mean TNR: 0.9256961292610913
Mean FPR: 0.07430387073890858
Mean TPR: 0.019883735906578694


In [6]:
# Initialize the Elastic Net attack
attack = BasicIterativeMethod(estimator=classifier, eps=0.01)

# Generate adversarial test examples
x_test_adv = attack.generate(x=x_test)

In [7]:
calculate_performance_metrics(x_test_adv, y_test, model)

4383/4383 [==============================] - 3s 632us/step
Accuracy: 0.9202952292662056
AUC: 0.994765402709281

macro
Precision: 0.8300630131547733
Recall: 0.6906494011023545
F1 Score: 0.7151242746981334

weighted
Precision: 0.9352760550594487
Recall: 0.9202952292662056
F1 Score: 0.9209862243781489

Mean FNR: 0.3093505988976454
Mean TNR: 0.9941571854952101
Mean FPR: 0.005842814504789895
Mean TPR: 0.6906494011023545


In [8]:
# Initialize the Elastic Net attack
attack = BasicIterativeMethod(estimator=classifier, eps=0.05)

# Generate adversarial test examples
x_test_adv = attack.generate(x=x_test)

In [9]:
calculate_performance_metrics(x_test_adv, y_test, model)

4383/4383 [==============================] - 3s 580us/step
Accuracy: 0.45160807245239964
AUC: 0.884119340969312

macro
Precision: 0.2558579609065153
Recall: 0.20851031821783167
F1 Score: 0.19057933865542012

weighted
Precision: 0.6020934861904934
Recall: 0.45160807245239964
F1 Score: 0.4979770074820043

Mean FNR: 0.7914896817821684
Mean TNR: 0.961032665857742
Mean FPR: 0.03896733414225808
Mean TPR: 0.20851031821783167
